In [ ]:
!pip install boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
!aws configure

In [ ]:
import requests
import os
import boto3
from botocore.exceptions import ClientError, NoCredentialsError
from botocore.config import Config

In [ ]:
S3_BUCKET = "s3-directed"

cities = {
    "Vienna": {
        "sea_mask": "s3://s3-directed/api_data/c5298b37096499d3c8bcfc49e449b393/seamask.tif",
        "dem_building": "s3://s3-directed/api_data/c5298b37096499d3c8bcfc49e449b393/dem_building.tif",
        "bbox": [16.1702, 48.0464, 16.6684, 48.3688]
    },
    "Esbjerg": {
        "sea_mask": "s3://s3-directed/api_data/b0d2dfae6d7dfe22594c58a28b00e183/seamask.tif",
        "dem_building": "s3://s3-directed/api_data/b0d2dfae6d7dfe22594c58a28b00e183/dem_building.tif",
        "bbox": [8.2858, 55.4121, 8.5328, 55.586]
    }
}


In [ ]:
# Flood map generation function with safer_rain model
def generate_safer_rain_flood_map(dem_file, output_file, s3_bucket, rain=0):

    # Define process request parameters
    url = "http://pygeoapi-saferplaces-lb-409838694.us-east-1.elb.amazonaws.com/processes/safer-rain-process/execution"


    s3_output_file = f"s3://{s3_bucket}/api_data/{output_file}"

    if rain:
        body = {
            "inputs": {
                "dem": dem_file,
                "rain": rain,
                "water": s3_output_file,
                "overwrite": True,
                "debug": True,
                "mode": "batch",
                "retries": 1,
                "timeout": 1200,
                "ram": 8192,
                "sync": True,
                "user": "saferplaces",
                "token": "S4fer_api_token"
            }
        }
        # Make the API request
        response = requests.post(url, json=body)
        if response.status_code == 200:
            print(response.json())
        else:
            print(f"Rain flood map generation failed: {response.text}")
            return None

        # Download the generated file from S3
        download_url = s3_output_file.replace("s3://", "https://s3.amazonaws.com/")
        output_local_path = os.path.basename(download_url)
        key_id = s3_output_file.split(f"s3://{s3_bucket}/")[-1]
        print(download_url)

        # Create an S3 client
        s3 = boto3.client('s3',config=Config(region_name="us-east-1"))

        # Download the file from S3
        try:
            # Retrieve the object
            s3.download_file(Bucket=s3_bucket,Key=key_id,Filename=output_file) # (Bucket=s3_bucket, Key="Rimini/wd.tif")
        except Exception as e:
            print(f"Error downloading file: {e}")

    else:
        print("No rain file provided.")
        return None


def generate_coastal_flood_map(file_dem, output_file, s3_bucket, file_seamask, file_barrier, esl):

    if output_file:
        s3_output_file = f"s3://{s3_bucket}/api_data/{output_file}"
    else:
        s3_output_file = None


    url = "http://pygeoapi-saferplaces-lb-409838694.us-east-1.elb.amazonaws.com/processes/safer-coast-process/execution"
    body = {
        "inputs": {
            "file_dem": file_dem,
            "file_seamask": file_seamask,
            "file_water": s3_output_file,
            "barrier": file_barrier,
            "esl": esl,
            'out_format': "COG",
            "token": "S4fer_api_token",
            "user": "saferplaces"
        }
    }
    # Make the API request
    response = requests.post(url, json=body)
    if response.status_code == 200:
        print(response.json())
        res_json = response.json()
    else:
        print(f"Coastal flood map generation failed: {response.text}")
        return None
    # Download the generated file from S3
    s3_output_file = res_json['files']['file_water']
    download_url = s3_output_file.replace("s3://", "https://s3.amazonaws.com/")
    output_local_path = os.path.basename(download_url)
    key_id = s3_output_file.split(f"s3://{s3_bucket}/")[-1]
    out_local_file = s3_output_file.rsplit("/",1)[1]
    if out_local_file == "water.tif":
        out_local_file = "coast_watermap.tif"
    print(download_url)

    # Create an S3 client
    s3 = boto3.client('s3',config=Config(region_name="us-east-1"))

    # Download the file from S3
    try:
        # Retrieve the object
        print(f"Downloading {key_id} from {s3_bucket}")
        s3.download_file(Bucket=s3_bucket,Key=key_id,Filename=out_local_file) # (Bucket=s3_bucket, Key="Rimini/wd.tif")
    except Exception as e:
        print(f"Error downloading file: {e}")


def main(requested_location,rain,esl=0,barrier=None,output_rain_file="rain_watermap.tif", output_coast_file=None,model="safer_rain",duration=3600, ti=600, tmax=3600, man=0.02, nl=25):

    if isinstance(rain,str):
        if not rain.endswith(".tif"):
            rain = float(rain)
    if isinstance(esl,str):
        esl = float(esl)

    dem_building = cities[requested_location]['dem_building']
    seamask = cities[requested_location]['sea_mask']

    if model == "safer_rain":

        print(f"Generating flood map with intensity of {rain}mm on {requested_location}.")
        generate_safer_rain_flood_map(rain=rain,s3_bucket=S3_BUCKET,dem_file=dem_building,output_file=output_rain_file)
        if esl != 0:
            print(f"Generating flood map with extreme sea level of {esl}m on {requested_location}")
            generate_coastal_flood_map(file_dem=dem_building,output_file=output_coast_file,s3_bucket=S3_BUCKET,file_seamask=seamask,file_barrier=barrier,esl=esl)
    return 0

In [ ]:
requested_location = "Esbjerg"
rain_intensity = "100"
extreme_sea_level = 3
main(requested_location,rain_intensity,esl=extreme_sea_level)

{'id': 'safer_coast_process', 'files': {'file_water': 's3://s3-directed/api_data/b0d2dfae6d7dfe22594c58a28b00e183/coast/f9a33307138e694fa275afce767a43a1/water.tif'}}
https://s3.amazonaws.com/s3-directed/api_data/b0d2dfae6d7dfe22594c58a28b00e183/coast/f9a33307138e694fa275afce767a43a1/water.tif


0